In [1]:
import pyspark

In [45]:
# Homework Question #5

pyspark.__version__

'3.3.4'

In [46]:
spark.version

'3.3.4'

In [3]:
pyspark.__file__

'/opt/bitnami/spark/python/pyspark/__init__.py'

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/04 16:19:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz

In [7]:
# !head fhv_tripdata_2019-10.csv.gz

In [ ]:
df = spark.read \
    .option("header", "true") \
    .option("compression", "gzip") \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [ ]:
df = spark.read.option("compression", 'gzip').csv('fhv_tripdata_2019-10.csv.gz', header=True, inferSchema=True)

In [ ]:
df.show()

In [ ]:
df.head(5)

In [ ]:
df.schema

In [ ]:
# Homework Question #2

df.repartition(6)
df.write.parquet('fhv_tripdata_2019-10/')

In [8]:
df = spark.read.parquet('fhv_tripdata_2019-10/')

In [9]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [10]:
from pyspark.sql import functions as F

In [11]:
df.dtypes

[('dispatching_base_num', 'string'),
 ('pickup_datetime', 'timestamp'),
 ('dropOff_datetime', 'timestamp'),
 ('PUlocationID', 'int'),
 ('DOlocationID', 'int'),
 ('SR_Flag', 'string'),
 ('Affiliated_base_number', 'string')]

In [12]:
df = df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropOff_datetime)) \
    # .show()

In [13]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|dropoff_date|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009| 2019-10-01|  2019-10-01|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013| 2019-10-01|  2019-10-01|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014| 2019-10-01|  2019-10-01|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014| 2019-10-01

In [19]:
# Homework Question #3
df \
    .filter(df.pickup_date == '2019-10-15') \
    .count() 

62610

In [30]:
# Homework Question #4
# option 1

df \
    .select(df.dropOff_datetime - df.pickup_datetime) \
    .sort(F.desc(df.dropOff_datetime - df.pickup_datetime)) \
    .show()
    

+------------------------------------+
|(dropOff_datetime - pickup_datetime)|
+------------------------------------+
|                INTERVAL '26298 0...|
|                INTERVAL '26298 0...|
|                INTERVAL '3653 00...|
|                INTERVAL '2922 00...|
|                INTERVAL '366 10:...|
|                INTERVAL '366 00:...|
|                INTERVAL '61 00:3...|
|                INTERVAL '44 00:4...|
|                INTERVAL '44 00:1...|
|                INTERVAL '33 01:3...|
|                INTERVAL '33 01:2...|
|                INTERVAL '33 01:1...|
|                INTERVAL '33 00:5...|
|                INTERVAL '33 00:5...|
|                INTERVAL '33 00:5...|
|                INTERVAL '33 00:4...|
|                INTERVAL '33 00:4...|
|                INTERVAL '33 00:4...|
|                INTERVAL '33 00:4...|
|                INTERVAL '33 00:4...|
+------------------------------------+
only showing top 20 rows



In [32]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|pickup_date|dropoff_date|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------+------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009| 2019-10-01|  2019-10-01|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013| 2019-10-01|  2019-10-01|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014| 2019-10-01|  2019-10-01|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014| 2019-10-01

In [34]:
# Homework Question #4
# option 2

df.registerTempTable('dhv_tripdata_2019_10')

df_largest_trip = spark.sql("""
SELECT
    max(dropOff_datetime - pickup_datetime) AS hour
FROM
    dhv_tripdata_2019_10
    

""")

In [35]:
df_largest_trip.show()

+--------------------+
|                hour|
+--------------------+
|INTERVAL '26298 0...|
+--------------------+



In [ ]:
# START
# Homework Question #6

In [ ]:
df_zones = spark.read.parquet('zones/')

In [37]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [38]:
df_zones.registerTempTable('zones')

In [43]:
df_least_frequent_pickup_zone = spark.sql("""

SELECT
    count(1) as count_pickup_zone,
    b.Zone
FROM
    dhv_tripdata_2019_10 a
LEFT JOIN
    zones b
    on a.PUlocationID = b.LocationID
GROUP BY
    b.Zone
ORDER BY
    count_pickup_zone asc


/*
SELECT
    min(count_pickup_zone),
    Zone
FROM
    (
        SELECT
            count(1) as count_pickup_zone,
            b.Zone
        FROM
            dhv_tripdata_2019_10 a
        LEFT JOIN
            zones b
            on a.PUlocationID = b.LocationID
        GROUP BY
            b.Zone
    )
GROUP BY
    Zone
ORDER BY    

*/


""")

In [44]:
# Homework Question #6 
# ANSWER
# Jamaica Bay
df_least_frequent_pickup_zone.show()

+-----------------+--------------------+
|count_pickup_zone|                Zone|
+-----------------+--------------------+
|                1|         Jamaica Bay|
|                2|Governor's Island...|
|                5| Green-Wood Cemetery|
|                8|       Broad Channel|
|               14|     Highbridge Park|
|               15|        Battery Park|
|               23|Saint Michaels Ce...|
|               25|Breezy Point/Fort...|
|               26|Marine Park/Floyd...|
|               29|        Astoria Park|
|               39|    Inwood Hill Park|
|               47|       Willets Point|
|               53|Forest Park/Highl...|
|               57|  Brooklyn Navy Yard|
|               62|        Crotona Park|
|               77|        Country Club|
|               89|     Freshkills Park|
|               98|       Prospect Park|
|              105|     Columbia Street|
|              110|  South Williamsburg|
+-----------------+--------------------+
only showing top

In [ ]:
!head -n 101 fhv_tripdata_2019-10.csv.gz > fhv_tripdata_2019-10_head.csv

In [ ]:
# pip install pandas

In [ ]:
import pandas as pd

In [ ]:
df_pandas = pd.read_csv('fhv_tripdata_2019-10.csv.gz', compression='gzip', nrows=100)

In [ ]:
df_pandas.shape[0]

In [ ]:
df_pandas.head(5)

In [ ]:
df_pandas.dtypes

In [ ]:
spark.createDataFrame(df_pandas).schema